# Problem Statement
Now that you have two new regression methods at your fingertips, it's time to give them a spin. In fact, for this challenge, let's put them together! Pick a dataset of your choice with a binary outcome and the potential for at least 15 features. If you're drawing a blank, the crime rates in 2013 dataset has a lot of variables that could be made into a modelable binary outcome.

Engineer your features, then create three models. Each model will be run on a training set and a test-set (or multiple test-sets, if you take a folds approach). The models should be:

Vanilla logistic regression
Ridge logistic regression
Lasso logistic regression
If you're stuck on how to begin combining your two new modeling skills, here's a hint: the SKlearn LogisticRegression method has a "penalty" argument that takes either 'l1' or 'l2' as a value.

In your report, evaluate all three models and decide on your best. Be clear about the decisions you made that led to these models (feature selection, regularization parameter selection, model evaluation criteria) and why you think that particular model is the best of the three. Also reflect on the strengths and limitations of regression as a modeling approach. Were there things you couldn't do but you wish you could have done?

### Outline
1. [Data](#data)
2. [Vanilla](#vanilla)
3. [Ridge](#ridge)
4. [Lasso](#lasso)
5. [Write-up](#write_up)

<a id = 'data'></a>
# Data

In [1]:
import pandas as pd
import numpy as np
current_state = np.random.get_state()
np.random.set_state(current_state)
from sklearn import linear_model
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.model_selection import cross_val_score

In [2]:
data_path = '../../datasets/fma/echonest_cols.csv'
df = pd.read_csv(data_path)
df_feat = df.copy()
display(df_feat.shape)

(13129, 250)

In [3]:
# unhelpful columns
df_feat.drop(columns = ['track_id', 'artist_name', 'release'], inplace = True)
# collinearity
temporal_cols = df_feat.columns[-224:]
feat_corr = df_feat.corr()
feat_corr = feat_corr.dropna(axis = 0)
corr_col = []
for col in temporal_cols:
    corrs = sorted(abs(feat_corr[col]), reverse = True)
    corr_score = corrs[0]+corrs[1]
    if corr_score > 1.5:
        corr_col.append(col)
# drop correlated columns
df_feat.drop(columns = corr_col, inplace = True)

# nans
df_nans = df_feat.isnull().sum()
nans_cols = list(df_nans[df_nans > 100].index)
df_feat.drop(columns = nans_cols, inplace = True)
df_feat.fillna(value = df_feat.median(), inplace = True)
# zeros
df_zeros = (df_feat == 0).sum()
zeros_cols = list(df_zeros[df_zeros > 100].index)
df_feat.drop(columns = zeros_cols, inplace = True)

In [4]:
# choose target and input
X = df_feat[list(set(list(df_feat.columns)) - set(['artist_familiarity']))]
y = df['artist_familiarity']
# y >= 0.3 is familiar, y < 0.3 is unfamiliar
y[y >= 0.3] = 1
y[y < 0.3] = 0

/home/conner/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/home/conner/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


<a id = 'vanilla'></a>
# Vanilla

In [5]:
def evaluate_model(model, X, y):
    # Fit the model.
    fit = model.fit(X, y)
    return cross_val_score(model, X, y, cv = 5), fit.coef_

In [13]:
vanilla_model = linear_model.LogisticRegression(C=1e9)
vanilla_scores, vanilla_coefs = evaluate_model(vanilla_model, X.apply(stats.zscore, axis = 1), y)

<a id = 'ridge'></a>
# Ridge

In [11]:
ridge_model = linear_model.Ridge(alpha = 1e-4)
ridge_scores, ridge_coefs = evaluate_model(ridge_model, X, y)
print (ridge_scores.mean(), ridge_scores.std())

0.5314933360005025 0.04496241681232213


<a id = 'lasso'></a>
# Lasso

In [12]:
lasso_model = linear_model.Lasso(alpha = 1e-7)
lasso_scores, lasso_coefs = evaluate_model(lasso_model, X, y)
print (lasso_scores.mean(), lasso_scores.std())

0.5314933257120503 0.044962440789950806


<a id = 'write_up'></a>
# Write-up

In [14]:
# display accuracy
print ("Vanilla\nMean: %0.2f (+/- %0.2f)" % (vanilla_scores.mean(), vanilla_scores.std()))
print ("Ridge\nMean: %0.2f (+/- %0.2f)" % (ridge_scores.mean(), ridge_scores.std()))
print ("Lasso\nMean: %0.2f (+/- %0.2f)" % (lasso_scores.mean(), lasso_scores.std()))

Vanilla
Mean: 0.87 (+/- 0.02)
Ridge
Mean: 0.53 (+/- 0.04)
Lasso
Mean: 0.53 (+/- 0.04)


In [15]:
# display coefficients
coefs = np.transpose([vanilla_coefs[0], ridge_coefs, lasso_coefs])
df_coefs = pd.DataFrame(data = coefs, index = X.columns, columns = ['vanilla', 'ridge', 'lasso'])
display(df_coefs)

,vanilla,ridge,lasso
169,0.668407,-0.000420,-0.000420
artist_discovery,-1295.552394,0.377661,0.377612
186,0.299681,-0.000674,-0.000674
170,0.498747,-0.000307,-0.000308
energy,8.214221,0.051143,0.051141
172,0.530788,0.005329,0.005329
211,2.986954,-0.000886,-0.000886
188,2.112576,0.007368,0.007368
acousticness,4.579229,0.049958,0.049957
174,-3.628858,-0.021206,-0.021206
